# Tabular Time Series Related API Demo with NOAA Weather Data

Copyright (c) Microsoft Corporation. All rights reserved. <br>
Licensed under the MIT License.

In this notebook, you will learn how to use the Tabular Time Series related API to filter the data by time windows for sample data uploaded to Azure blob storage. 

The detailed APIs to be demoed in this script are:
- Create Tabular Dataset instance
- Assign timestamp column and partition timestamp column for Tabular Dataset to activate Time Series related APIs
- Clear timestamp column and partition timestamp column
- Filter in data before a specific time
- Filter in data after a specific time
- Filter in data in a specific time range
- Filter in data for recent time range

Besides above APIs, you'll also see:
- Create and load a Workspace
- Load weather data into Azure blob storage
- Create and register weather data as a Tabular dataset
- Re-load Tabular Dataset from your Workspace

## Import Dependencies

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, run the cells below to install the Azure Machine Learning Python SDK and create an Azure ML Workspace that's required for this demo.

## Prepare Environment

Print out your version of the Azure ML Python SDK. Version 1.0.60 or above is required for TabularDataset with timeseries attribute. 

In [ ]:
import azureml.core
azureml.data.__version__

## Import Packages

In [ ]:
# import packages
from datetime import datetime, timedelta

from azureml.core import Dataset, Workspace

## Set up Configuraton and Create Azure ML Workspace

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) first if you haven't already to establish your connection to the Azure ML Workspace.

In [ ]:
ws = Workspace.from_config()
dstore = ws.get_default_datastore()

dset_name = 'weather-data-florida'

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, dstore.name, sep = '\n')

## Load Data to Blob Storage

This demo uses 2019 weather data under within weather-data folder. You can replace this data with your own.

Upload data to blob storage so it can be used as a Dataset.

In [ ]:
dstore.upload('weather-data', dset_name, overwrite=True, show_progress=True)

## Create & Register Tabular Dataset with time-series trait from Blob

The API on Tabular datasets with time-series trait is specially designed to handle Tabular time-series data and time related operations more efficiently. By registering your time-series dataset, you are publishing your dataset to your workspace so that it is accessible to anyone with the same subscription id. 

Create Tabular Dataset instance from blob storage datapath.

**TIP:** you can set virtual columns in the partition_format. I.e. if you partition the weather data by state and city, the path can be '/{STATE}/{CITY}/{partition_time:yyy/MM}/data.parquet'. STATE and CITY would then appear as virtual columns in the dataset, allowing for efficient filtering by these timestamps. 

In [ ]:
datastore_path = [(dstore, dset_name + '/*/*/data.parquet')]
dataset        = Dataset.Tabular.from_parquet_files(path=datastore_path, partition_format = dset_name + '/{partition_time:yyyy/MM}/data.parquet')

Assign "datetime" column as timestamp and "partition_time" from folder path as partition_timestamp for Tabular Dataset to activate Time Series related APIs. The column to be assigned should be a Date type, otherwise the assigning will fail.

In [ ]:
tsd = dataset.with_timestamp_columns(timestamp='datetime', partition_timestamp='partition_time')

Register the dataset for easy access from anywhere in Azure ML and to keep track of versions, lineage. 

In [ ]:
# register dataset to Workspace
registered_ds = tsd.register(ws, dset_name, create_new_version=True, description='Data for Tabular Dataset with time-series trait demo.', tags={ 'type': 'TabularDataset' })

## Reload the Dataset from Workspace

In [ ]:
# get dataset by dataset name
tsd = Dataset.get_by_name(ws, name=dset_name)
tsd.to_pandas_dataframe().head(5)

## Filter Data by Time Windows

Once your data has been loaded into the notebook, you can query by time using the time_before(), time_after(), time_between(), and time_recent() functions.The filter is optimized to only load those data files within the partition_timestamp range when partition_timestamp is specified.

include_boundary is default to be true for all the time series related filters, please pass include_boundary=False to exclude boundary.

### Before Time Input

In [ ]:
# select data that occurs before a specified date
tsd2 = tsd.time_before(datetime(2019, 6, 12))
tsd2.to_pandas_dataframe().tail(5)

## After Time Input

In [ ]:
# select data that occurs after a specified date
tsd2 = tsd.time_after(datetime(2019, 5, 30))
tsd2.to_pandas_dataframe().head(5)

## Before & After Time Inputs

You can chain time functions together.

In [ ]:
# select data that occurs within a given time range
tsd2 = tsd.time_after(datetime(2019, 1, 1)).time_before(datetime(2019, 1, 10))
tsd2.to_pandas_dataframe().head(5)

### Time Range Input

In [ ]:
# another way to select data that occurs within a given time range
tsd2 = tsd.time_between(start_time=datetime(2019, 1, 31, 23, 59, 59), end_time=datetime(2019, 2, 7))
tsd2.to_pandas_dataframe().head(5)

## Time Recent Input

This function takes in a datetime.timedelta and returns a dataset containing the data from datetime.now()-timedelta() to datetime.now().

**NOTE:** This will return an empty dataframe there is no data within the last 2 days.

In [ ]:
tsd2 = tsd.time_recent(timedelta(days=2))
tsd2.to_pandas_dataframe().tail(5)

## Drop and keep columns

You can also choose to drop or keep certain columns.

### Drop Columns

<font color=red>If a timeseries column is dropped, the corresponding capabilities will be dropped for the returned dataset.</font><br>

In [ ]:
tsd2 = tsd.drop_columns(columns=['snowDepth', 'version', 'datetime'])
tsd2.take(5).to_pandas_dataframe()

The exception is expected because dataset loses timeseries capabilities to do time travel.

In [ ]:
from azureml.exceptions import DatasetTimestampMissingError

try:
    tsd2.time_before(datetime(2019, 6, 12)).to_pandas_dataframe().tail(5)
except DatasetTimestampMissingError as e:
    print('Expected exception : {}'.format(str(e)))

Drop will return dataset with timeseries capabilities if modify column list to exclude timestamp columns.

In [ ]:
tsd2 = tsd.drop_columns(columns=['snowDepth', 'version', 'upload_date'])
tsd2.take(5).to_pandas_dataframe()

In [ ]:
tsd2.time_before(datetime(2019, 6, 12)).to_pandas_dataframe().tail(5)

### Keep Columns

<font color=red>If a timeseries column is not included, the timeseries capabilities will be dropped for the returned dataset.</font><br>

In [ ]:
tsd2 = tsd.keep_columns(columns=['snowDepth'], validate=False)
tsd2.to_pandas_dataframe().tail()

The exception is expected because dataset loses timeseries capabilities to do time travel.

In [ ]:
try:
    tsd2.time_before(datetime(2019, 6, 12)).to_pandas_dataframe().tail(5)
except DatasetTimestampMissingError as e:
    print('Expected exception : {}'.format(str(e)))

Keep will return dataset with timeseries capabilities if modify column list to include timestamp columns.

In [ ]:
tsd2 = tsd.keep_columns(columns=['snowDepth', 'datetime', 'partition_time'], validate=False)
tsd2.to_pandas_dataframe().tail()

In [ ]:
tsd2.time_before(datetime(2019, 6, 12)).to_pandas_dataframe().tail(5)

## Resetting Timestamp Columns

Rules for reseting are:
- You cannot assign 'None' to timestamp while assign a valid column name to partition_timestamp because partition_timestamp is optional while timestamp is mandatory for Tabular time series data.
- If you assign 'None' to timestamp, then both timestamp and partition_timestamp will all be cleared.
- If you assign only 'None' to partition_timestamp, then only partition_timestamp will be cleared.

In [ ]:
from azureml.exceptions import UserErrorException
# Illegal clearing, exception is expected.
try:
    tsd2 = tsd.with_timestamp_columns(timestamp=None, partition_timestamp='partition_time')
except UserErrorException as e:
    print('Cleaning not allowed because {}'.format(str(e)))

# clear both
tsd2 = tsd.with_timestamp_columns(timestamp=None, partition_timestamp=None)
print('after clean both with None/None, timestamp columns are: {}'.format(tsd2.timestamp_columns))

# clear partition_timestamp only and assign 'datetime' as timestamp column
tsd2 = tsd2.with_timestamp_columns(timestamp='datetime', partition_timestamp=None)
print('after clean partition timestamp column, timestamp columns are: {}'.format(tsd2.timestamp_columns))

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/datasets-tutorial/datasets-tutorial.png)